<a href="https://colab.research.google.com/github/greentea8610/Video-Clipping/blob/main/youtube_dl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

sources: 
[clip_specific_time](https://stackoverflow.com/questions/57131049/is-it-possible-to-download-a-specific-part-of-a-file),
[yt-dlc](https://github.com/blackjack4494/yt-dlc#installation)


if you experience any issues, ping me on Discord @Greentea#8610

# Run
You don't need to touch this. You can just click on the run button (play button)

In [ ]:
# A prompt will come up asking you to continue and you can jsut press enter
!pip install youtube_dl
!apt install ffmpeg
!pip install --upgrade youtube-dlc

     |████████████████████████████████| 1.9MB 5.4MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.
     |████████████████████████████████| 1.8MB 5.8MB/s 


In [ ]:
import os, sys, datetime, shutil
from google.colab import drive
import re
import youtube_dl, subprocess
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def get_duration(FROM, TO):
    TO = "00:"+TO if len(TO) == 5 else "00:0"+TO if len(TO) == 4 else "0"+TO if len(TO) == 7 else TO
    FROM = "00:"+FROM if len(FROM) == 5 else "00:0"+FROM if len(FROM) == 4 else "0"+FROM if len(FROM) == 7 else FROM

    duration = (pd.to_datetime(TO) - pd.to_datetime(FROM))
    days, seconds = duration.days, duration.seconds
    hours = "0"+str(days * 24 + seconds // 3600) if len(str(days * 24 + seconds // 3600)) == 1 else str(days * 24 + seconds // 3600)
    minutes = "0"+str((seconds % 3600) // 60) if len(str((seconds % 3600) // 60)) == 1 else str((seconds % 3600) // 60)
    seconds = "0"+str(seconds % 60) if len(str(seconds % 60)) == 1 else str(seconds % 60)
    return hours+":"+minutes+":"+seconds

In [ ]:
def clean_timestamp(timestamp_val, timestamp_type, add_pad, verbose=False):
    if verbose == True:
        print(timestamp_val)
    timestamp_val = "00:"+timestamp_val if len(timestamp_val) == 5 else "00:0"+timestamp_val if len(timestamp_val) == 4 else "0"+timestamp_val if len(timestamp_val) == 7 else timestamp_val
    
    if timestamp_type == "end_time":
        adjusted_timestamp = pd.to_datetime(timestamp_val) + timedelta(seconds=add_pad)
    elif timestamp_type == "start_time":
        if timestamp_val == "00:00:00":
            adjusted_timestamp = pd.to_datetime(timestamp_val)
        else:
            adjusted_timestamp = pd.to_datetime(timestamp_val) - timedelta(seconds=add_pad)
    adjusted_timestamp = (adjusted_timestamp - datetime.today().replace(hour=0, minute=0, second=0, microsecond=0))

    days, seconds = adjusted_timestamp.days, adjusted_timestamp.seconds
    hours = "0"+str(days * 24 + seconds // 3600) if len(str(days * 24 + seconds // 3600)) == 1 else str(days * 24 + seconds // 3600)
    minutes = "0"+str((seconds % 3600) // 60) if len(str((seconds % 3600) // 60)) == 1 else str((seconds % 3600) // 60)
    seconds = "0"+str(seconds % 60) if len(str(seconds % 60)) == 1 else str(seconds % 60)
    return hours+":"+minutes+":"+seconds

In [ ]:
def get_clip(URL, FROM, TO, TARGET=[], format="best", method="ytdl", folder=""):
    duration = get_duration(FROM, TO)

    with youtube_dl.YoutubeDL({'format': format, 'cachedir': False}) as ydl:
        result = ydl.extract_info(URL, download=False)
        video = result['entries'][0] if 'entries' in result else result
        if TARGET is get_clip.__defaults__[0]:
            TARGET = result["title"]+"_"+FROM+"_"+TO+".mp4"
    url_value = '"'+str(video['url'])+'"'

    if method == "ytdl":
        !youtube-dl -q -f bestvideo+bestaudio --add-metadata --write-thumbnail --write-description --write-info-json --hls-use-mpegts --no-part $URL -o clip
        !ffmpeg -i clip.webm -c:v copy -c:a copy $TARGET
        !ffmpeg -i clip.mkv -c:v copy -c:a copy $TARGET

    if method == "ytdlc":
        raise Exception("This feature is still a WIP.")
        !youtube-dlc -q -f bestvideo+bestaudio --add-metadata --write-thumbnail --write-description --write-info-json --hls-use-mpegts --no-part $URL -o $TARGET

    if method == "ffmpeg":  
        !ffmpeg -y -ss $FROM -i $url_value -t $duration $TARGET

    if folder != "":
        !cp $TARGET drive/My\ Drive/$FOLDER/$TARGET

In [ ]:
def aonohara_get_clip(youtube_url, clip_start_time, duration, filename, folder=""):
    if folder=="":
        absolute_drive_path = filename
    else:
        absolute_drive_path = os.path.join("/content/drive/My Drive", folder, filename)
    
    mount_drive = True
    if mount_drive:
        drive.mount('/content/drive')

    os.environ['YT_URL'] = youtube_url
    os.environ['START_TIME'] = clip_start_time
    os.environ['LENGTH'] = duration
    os.environ['FILENAME'] = filename
    os.environ['RAWVIDEO'] = filename[:-4]
    !youtube-dl "$YT_URL" --output "$RAWVIDEO"
    !ffmpeg -y -i "$(ls $RAWVIDEO.*)" -ss "$START_TIME" -t "$LENGTH" -c:v libx264 -c:a aac "$FILENAME"
    
    if folder!="":
        print("Copying file to Google Drive...")
        shutil.copy2("/content/"+filename, absolute_drive_path)
        print("File copied to Google Drive at", absolute_drive_path)
        !rm "$FILENAME"
        !rm "$(ls $RAWVIDEO.*)"

# Config

## Download Video

#### for one-time downloads

In [ ]:
# url of the video you want to clip
# URL = "https://www.youtube.com/watch?v=zHODWkeNMYA" # Kanata GTA V
URL = "https://www.youtube.com/watch?v=KRMfnvLODZQ"
18, 21, 29, 20
# [5:27:43-5:27:48], [5:34:29-5:34:37], [5:39:45-5:40:01], [11:50:26-11:50:36]
# start time of the clip
FROM = "11:50:21"
# end time of the clip
TO = "11:50:41"

# filename of the output file
# TARGET = os.path.join(os.getcwd(),"drive","My\ Drive","Doris","Raw\ Clips","Clip10.mkv")
FOLDER = ""
TARGET = "Clip11_4.mp4"

# get the seconds conversion of FROM
FROM_seconds = int((datetime.strptime(FROM, "%H:%M:%S") - datetime(1900, 1, 1)).total_seconds())

# YouTubeVideo(URL.split("=")[1], start = FROM_seconds, autoplay=1, theme="light", color="red")

In [ ]:
# get_clip(URL, FROM, TO, TARGET, method="ffmpeg")
aonohara_get_clip(URL, FROM, get_duration(FROM, TO), TARGET, FOLDER)
# !ffmpeg -i clip.mkv -c:v copy -c:a copy $TARGET

[download] 100% of 20.18GiB in 09:13
[download] Destination: Clip11_3.f251
[download] 100% of 20.18GiB in 09:13
[download] Destination: Clip11_3.f251
[download] 100% of 539.30MiB in 00:14
[ffmpeg] Merging formats into "Clip11_3.mkv"
[download] 100% of 539.30MiB in 00:14
[ffmpeg] Merging formats into "Clip11_3.mkv"
Deleting original file Clip11_3.f299 (pass -k to keep)
Deleting original file Clip11_3.f299 (pass -k to keep)
Deleting original file Clip11_3.f251 (pass -k to keep)
Deleting original file Clip11_3.f251 (pass -k to keep)
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-lib

#### for multiple downloads
if you have multiple clips you need to download, you can place the data in a google sheet.

The sheet must have a column named Link & Timestamp containing the url and timestamps you want to parse respectively.

The Timestamp column must follow the format [hh:mm:ss-hh:mm:ss], [hh:mm:ss-hh:mm:ss]

In [ ]:
!pip install --upgrade gspread
from google.colab import auth
import gspread
from oauth2client.client import GoogleCredentials
from tqdm import tqdm

  Found existing installation: gspread 3.0.1
    Uninstalling gspread-3.0.1:
      Successfully uninstalled gspread-3.0.1


In [ ]:
def extract_timestamps(sheet, sheet_name, add_pad,verbose=False):
    df = pd.DataFrame(sheet.get_all_records()).rename(columns={"Timestamps":"Timestamp", "Time Stamps":"Timestamp", "What's happening":"Desc", "Explanation":"Desc", "YouTube Link":"Link"})
    df_sliced = df[(df["Filename"]!="")].reset_index(drop=True)
    df_sliced["Timestamp"] = df_sliced["Timestamp"].str.strip()
    url = [df_sliced.loc[row, "Link"][:43] for row in range(len(df_sliced))]
    timestamp_list = [re.sub("[^0-9:,-]", "", df_sliced.loc[row, "Timestamp"]).split(",") for row in range(len(df_sliced))]
    timestamp_series = pd.Series(timestamp_list).apply(pd.Series).reset_index().melt(id_vars='index').dropna()[['index', 'value']].set_index('index')
    df_merge = pd.merge(df_sliced, timestamp_series, left_index=True, right_index=True).reset_index(drop=True)
    df_merge["start_time"] = df_merge["value"].str.split("-", expand=True)[0]
    df_merge["Link"] = df_merge["Link"].apply(lambda x: x.split("?")[0] if "youtu.be" in x else x.split("&")[0])
    df_merge["end_time"] = df_merge["value"].str.split("-", expand=True)[1]
    df_merge["start_time_new"] = df_merge["start_time"].apply(lambda x: clean_timestamp(x,"start_time", add_pad, verbose))
    df_merge["end_time_new"] = df_merge["end_time"].apply(lambda x: clean_timestamp(x,"end_time", add_pad, verbose))
    df_merge["duration"] = df_merge[["start_time_new", "end_time_new"]].apply(lambda x : get_duration(*x), axis=1)
    # df_merge["gap"] = df_merge[["start_time_new", "end_time_new"]]
    df_merge["count"] = df_merge.groupby('Desc').cumcount() + 1
    df_merge["folder"] = df_merge["Desc"].apply(lambda x: re.sub("[^A-Za-z0-9 ._]", "", x)).str.replace(" ","_")
    df_merge["title"] = df_merge["Desc"].apply(lambda x: re.sub("[^A-Za-z0-9 ._]", "", x+"_Clip")).str.replace(" ","_")
    df_merge["title"] = df_merge["title"] + df_merge["count"].apply(str) + ".mp4"
    df_merge["sheet_name"] = sheet_name
    return df_merge.drop(["count"], axis=1)

In [ ]:
def get_sheet_time(url, sheets_list=[], verbose=False):

    auth.authenticate_user()
    gc = gspread.authorize(GoogleCredentials.get_application_default())
    df = pd.DataFrame()

    worksheet = gc.open_by_url(url)

    for sheet in worksheet.worksheets():
        if sheet.title in sheets_list:
            if verbose == True:
                print(sheet.title)
            placeholder_df = extract_timestamps(sheet, sheet.title, verbose)
            df = pd.concat([df, placeholder_df])
    return df.drop(["value"], axis=1).reset_index(drop=True)

In [ ]:
url = "https://docs.google.com/spreadsheets/d/1mg18bJE5J1qHGejVjKRx9ZUBUCbaEdWYSPCLy9SyA2s/edit#gid=1909165107"

sheets_list = ["Doris"]
failed_list = []
interval = 0

## No need to touch the code beyond this line.
df = get_sheet_time(url, sheets_list, interval)#, verbose=True)
print(df.shape)
df

# sheet_name = df.loc[0, "sheet_name"].split(" ")[1].replace(" ", "\ ")
# for folder in df.folder.unique():
#     !mkdir -p /content/drive/My\ Drive/$sheet_name/Raw\ Clips/$folder

# for row in range(31, 32):
#     print(df.loc[row, "Link"], df.loc[row, "start_time_new"], df.loc[row, "end_time_new"], df.loc[row, "duration"], df.loc[row, "sheet_name"].split(" ")[-1], df.loc[row, "folder"], df.loc[row, "title"])
    # try:
    #     aonohara_get_clip(df.loc[row, "Link"], df.loc[row, "start_time_new"], df.loc[row, "duration"], df.loc[row, "sheet_name"].split(" ")[-1], df.loc[row, "folder"], df.loc[row, "title"])
    #     # get_clip(df.loc[row, "Link"], df.loc[row, "start_time_new"], df.loc[row, "end_time_new"], "/content/sample/"+df.loc[row, "folder"]+"/"+df.loc[row, "title"])
    # except Exception as e:
    #     failed_list.append(df.loc[row, "Link"])
        # print(e, df.loc[row, "Link"])

(11, 17)


,Clip number,Date,Link,Timestamp:,Lengths,Desc,Final order,Filename,Timestamp,start_time,end_time,start_time_new,end_time_new,duration,folder,title,sheet_name
0,1,2020-07-14,https://www.bilibili.com/video/BV1jC4y1h79T/,[1:00-1:17],17,Doris gets bombarded by SCs.,,Clip1,[1:00-1:17],1:00,1:17,00:01:00,00:01:17,00:00:17,Doris_gets_bombarded_by_SCs.,Doris_gets_bombarded_by_SCs._Clip1.mp4,Doris
1,2,2020-04-25,https://www.bilibili.com/video/BV1V64y1M7UV,[0:32-1:11],39,"Start Dash, one of the most famous songs in LL...",,Clip2,[0:32-1:11],0:32,1:11,00:00:32,00:01:11,00:00:39,Start_Dash_one_of_the_most_famous_songs_in_LL....,Start_Dash_one_of_the_most_famous_songs_in_LL....,Doris
2,3,2020-04-14,https://www.bilibili.com/video/BV1ga4y147Ni,[118:12-119:21],69,"Thank you friends, very emotional song.",,Clip3,[01:58:12-01:59:21],01:58:12,01:59:21,01:58:12,01:59:21,00:01:09,Thank_you_friends_very_emotional_song.,Thank_you_friends_very_emotional_song._Clip1.mp4,Doris
3,4,2020-03-15,https://www.bilibili.com/video/BV1KE41137Ja,[1:04-1:32],28,"Doris is a mermaid+Debut vid, shinkai shoujo r...",,Clip4,[1:04-1:32],1:04,1:32,00:01:04,00:01:32,00:00:28,Doris_is_a_mermaidDebut_vid_shinkai_shoujo_res...,Doris_is_a_mermaidDebut_vid_shinkai_shoujo_res...,Doris
4,5,2020-08-13,https://www.bilibili.com/video/BV165411874h,[0:00-0:10],10,Doris is fooood,,Clip5,[0:00-0:10],0:00,0:10,00:00:00,00:00:10,00:00:10,Doris_is_fooood,Doris_is_fooood_Clip1.mp4,Doris
5,6,2020-06-02,https://www.bilibili.com/video/BV13k4y1z72Z,[19:38-20:17],39,Emotional Birthday moment,,Clip6,[19:38-20:17],19:38,20:17,00:19:38,00:20:17,00:00:39,Emotional_Birthday_moment,Emotional_Birthday_moment_Clip1.mp4,Doris
6,7,2020-07-19,https://www.bilibili.com/video/BV1rv411q7sF/,[0:26-0:44],18,Doris getting baited by 98K in pubg,,Clip7,[0:26-0:44],0:26,0:44,00:00:26,00:00:44,00:00:18,Doris_getting_baited_by_98K_in_pubg,Doris_getting_baited_by_98K_in_pubg_Clip1.mp4,Doris
7,8,2020-07-17,https://www.bilibili.com/video/BV1u5411a7d5,[2:50-2:54],4,Doris-chan wa kyou mo kawaii,,Clip8,[2:50-2:54],2:50,2:54,00:02:50,00:02:54,00:00:04,Dorischan_wa_kyou_mo_kawaii,Dorischan_wa_kyou_mo_kawaii_Clip1.mp4,Doris
8,9,2020-06-05,https://www.bilibili.com/video/BV1tK411p7WY,[0:59-1:59],60,Doris confessing to Rosalyn,,Clip9,[0:59-1:59],0:59,1:59,00:00:59,00:01:59,00:01:00,Doris_confessing_to_Rosalyn,Doris_confessing_to_Rosalyn_Clip1.mp4,Doris
9,10,2020-04-05,https://www.bilibili.com/video/BV1RT4y137Xm,[62:10 - 62:22],12,Start of Doris bond with fans,,Clip10,[1:02:10 - 1:02:22],1:02:10,1:02:22,01:02:10,01:02:22,00:00:12,Start_of_Doris_bond_with_fans,Start_of_Doris_bond_with_fans_Clip1.mp4,Doris


In [ ]:
df_copy = df.copy()
df_copy["start_time"] = df_copy["start_time"].apply(lambda x: "["+x)
df_copy["start_time_new"] = df_copy["start_time_new"].apply(lambda x: "["+x)
df_copy["end_time"] = df_copy["end_time"].apply(lambda x: "-"+x+"]")
df_copy["end_time_new"] = df_copy["end_time_new"].apply(lambda x: "-"+x+"]")
df_copy["final_time"] = df_copy["start_time_new"] + df_copy["end_time_new"]
df_copy["Timestamp"] = df_copy["start_time"] + df_copy["end_time"]
df_copy["Filename"] = df_copy["Filename"].apply(lambda x: "-c copy drive/My\ Drive/"+df.loc[0, "sheet_name"].split(" ")[-1]+"/Raw\ Clips/"+x+".mkv")
df_copy_grouped = df_copy.groupby(["Filename", "Link", "folder"])[["final_time", "Timestamp"]].sum().reset_index()
df_copy_grouped.head(2)

df_copy_grouped[["Link","final_time","Filename","Timestamp","folder"]].to_csv(df.loc[0, "sheet_name"].split(" ")[-1]+"_"+str(interval)+"secpad.csv", index=False)

# Old Scripts

## Play Video

In [ ]:
!ffprobe "Graduation Ceremony - Wanna be your sunshine-hceZWCwcyBs.mp4"

In [ ]:
from IPython.display import HTML, YouTubeVideo
from base64 import b64encode

mp4 = open('Graduation Ceremony - Wanna be your sunshine-hceZWCwcyBs.mp4','rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML("""
<video width=720 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

## Move Video

In [ ]:
import os
master_clips_folder = os.path.join(os.getcwd(), "drive", "My Drive", "Hololive Resistance Youtube", "Collab", "2020 Year HoloRewind (w  Aonahara + HLR)", "Master Assets")
miyabi_folder = os.path.join(master_clips_folder, "Stars Gen 1", "Miyabi", "Raw Clips")
miyabi_folder

'/content/drive/My Drive/Hololive Resistance Youtube/Collab/2020 Year HoloRewind (w  Aonahara + HLR)/Master Assets/Stars Gen 1/Miyabi/Raw Clips'

In [ ]:
!gsutil cp -r /content/sample/ /content/drive/My\ Drive/Hololive\ Resistance\ Youtube/Collab/2020\ Year\ HoloRewind\ \(w\ \ Aonahara\ +\ HLR\)/Master\ Assets/Stars\ Gen\ 1/Miyabi/Raw\ Clips

Copying file:///content/sample/Miyabi_got_scared_with_loading_screen_voice/Miyabi_got_scared_with_loading_screen_voice_Clip_1.mp4...
Copying file:///content/sample/Oran_ge_Orange_/Oran_ge_Orange__Clip_2.mp4...
Copying file:///content/sample/Oran_ge_Orange_/Oran_ge_Orange__Clip_1.mp4...
Copying file:///content/sample/Miyabi_got_a_hair_strand_on_chocolate/Miyabi_got_a_hair_strand_on_chocolate_Clip_2.mp4...
/ [4 files][  8.0 MiB/  8.0 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file:///content/sample/Miyabi_got_a_hair_strand_on_chocolate/Miyabi_got_a_hair_strand_on_chocolate_Clip_4.mp4...
Copying file:///content/sample/Miyabi_got_a_hair_strand_on_chocolate/Miyabi_got_a_hair_strand_on_chocolate_Clip_3.mp4...
Copying file:///c

## Create Black Screen Videos

root_folder

In [ ]:
multipleclips_folders_list = [folder for folder in df["folder"].unique() if len(os.listdir(os.path.join(root_folder, folder))) > 1]
multipleclips_folders_output = list(df[df['folder'].isin(multipleclips_folders_list)]["Filename"].unique())
singleclips_folders_list = [folder for folder in df["folder"].unique() if len(os.listdir(os.path.join(root_folder, folder))) == 1]
singleclips_folders_output = list(df[df['folder'].isin(singleclips_folders_list)]["Filename"].unique())
black_files_list = [os.path.join(root_folder, folder, file).replace(".mp4","andblack.mp4").replace(" ", "\ ") for folder in df["folder"].unique() for file in os.listdir(os.path.join(root_folder, folder)) if len(os.listdir(os.path.join(root_folder, folder))) > 0]
files_list = [file.replace(" ", "\ ") for folder in df["folder"].unique() for file in os.listdir(os.path.join(root_folder, folder)) if len(os.listdir(os.path.join(root_folder, folder))) > 0]
files_list

In [ ]:
for file in files_list[:6]:
    os.environ["FILE"] = file
    os.environ["NEW_FILE"] = file.replace(".mp4", "_new.mp4")
    !ffmpeg -i "$FILE" "$NEW_FILE"

In [ ]:
for file in black_files_list[:6]:
    # !ffmpeg -f lavfi -i color=black:s=1920x1080:r=24000/1001 -f lavfi -i anullsrc -ar 44100 -ac 128 -t 3 $file
    !ffmpeg -y -f lavfi -i color=c=black:s=1920x1080:d=3 -f lavfi -i aevalsrc=0:c=2:d=3 $file
    # !ffmpeg -f lavfi -i color=c=black:s=640x480:d=2 -f lavfi -i aevalsrc=0:c=2:d=2 black_screen_video.mp4

## Merge Videos

In [ ]:
%cd /content/drive/My\ Drive/Master\ Assets/Stars\ Gen\ 1/Miyabi/Raw\ Clips/old/Miyabis_is_a_human
!pwd
# !for f in ./*.mp4; do echo "file '$f'" >> mylist.txt; done
# !ffmpeg -y -f concat -safe 0 -i mylist.txt Clip16.mkv
!ffmpeg -i Clip13.mp4 -codec copy Clip13.mkv

# for folder in multipleclips_folders_list[:1]:
#     os.environ["FOLDER"] = folder
#     %cd /content/drive/My\ Drive/Master\ Assets/Stars\ Gen\ 1/Miyabi/Raw\ Clips/$folder
#     !pwd
#     !for f in ./*.mp4; do echo "file '$f'" >> mylist.txt; done

/content/drive/.shortcut-targets-by-id/113/Master Assets/Stars Gen 1/Miyabi/Raw Clips/old/Miyabis_is_a_human
/content/drive/.shortcut-targets-by-id/113/Master Assets/Stars Gen 1/Miyabi/Raw Clips/old/Miyabis_is_a_human
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enabl

In [ ]:
for folder, file in zip(multipleclips_folders_list[:1], multipleclips_folders_output[:1]):
    os.environ["FILE"] = file+".mkv"
    os.environ["FOLDER"] = folder
    %cd /content/drive/My\ Drive/Master\ Assets/Stars\ Gen\ 1/Miyabi/Raw\ Clips/$folder
    !pwd
    !ffmpeg -y -f concat -safe 0 -i mylist.txt "$FILE"

### Extra code

In [ ]:
!ffmpeg -f lavfi -i color=size=1280x720:rate=1187:color=black -f lavfi -i anullsrc=channel_layout=stereo:sample_rate=48000 -t 3 Is_that_YAGOO_Clip1_black.mp4

In [ ]:
!ffmpeg -f concat -safe 0 -i mylist.txt -c copy clip1.mkv

In [ ]:
!ffprobe Is_that_YAGOO_Clip1_black.mp4

ffprobe version 3.4.8-0ubuntu0.2 Copyright (c) 2007-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-li

In [ ]:
!ffprobe Is_that_YAGOO_Clip1.mp4

ffprobe version 3.4.8-0ubuntu0.2 Copyright (c) 2007-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-li

In [ ]:
!curl https://temp-clipping-tool.vercel.app/api/colab/eyJ1cmwiOiJodHRwczovL3d3dy55b3V0dWJlLmNvbS93YXRjaD92PS14UjVBbW10VXI0IiwidGltZXN0YW1wcyI6IlsxMToyNy0xMTo0MF0sWzExOjUxLTExOjU1XSIsIm91dHB1dCI6Ii1jOnYgbGlieDI2NCAtYzphIGFhYyAtcGl4X2ZtdCB5dXY0MjBwIG91dHB1dC5ta3YifQ%3D%3D | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   836  100   836    0     0    368      0  0:00:02  0:00:02 --:--:--   368
--2020-11-11 13:54:36--  http://youtube-dl.org/downloads/latest/youtube-dl
Resolving youtube-dl.org (youtube-dl.org)... 95.143.172.170, 2001:1a50:11:0:5f:8f:acaa:177
Connecting to youtube-dl.org (youtube-dl.org)|95.143.172.170|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://youtube-dl.org/downloads/2020.11.01.1/youtube-dl [following]
--2020-11-11 13:54:37--  http://youtube-dl.org/downloads/2020.11.01.1/youtube-dl
Connecting to youtube-dl.org (youtube-dl.org)|95.143.172.170|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://gitlab.com/dstftw/youtube-dl/uploads/7c81fe65744e2304de741d54f989093a/youtube-dl [following]
--2020-11-11 13:54:37--  https://gitlab.com/dstftw/youtube-dl/uploads

In [ ]:
!curl https://temp-clipping-tool.vercel.app/api/colab/eyJ1cmwiOiJodHRwczovL3d3dy55b3V0dWJlLmNvbS93YXRjaD92PS1OelEwdmdHY1JjIiwidGltZXN0YW1wcyI6IlsyOjE5LTI6MzNdIiwib3V0cHV0IjoiLWM6diBsaWJ4MjY0IC1jOmEgYWFjIC1waXhfZm10IHl1djQyMHAgY2xpcDYubWt2In0%3D | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   645  100   645    0     0    479      0  0:00:01  0:00:01 --:--:--   479
--2020-11-11 14:12:25--  http://youtube-dl.org/downloads/latest/youtube-dl
Resolving youtube-dl.org (youtube-dl.org)... 95.143.172.170, 2001:1a50:11:0:5f:8f:acaa:177
Connecting to youtube-dl.org (youtube-dl.org)|95.143.172.170|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://youtube-dl.org/downloads/2020.11.01.1/youtube-dl [following]
--2020-11-11 14:12:25--  http://youtube-dl.org/downloads/2020.11.01.1/youtube-dl
Connecting to youtube-dl.org (youtube-dl.org)|95.143.172.170|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://gitlab.com/dstftw/youtube-dl/uploads/7c81fe65744e2304de741d54f989093a/youtube-dl [following]
--2020-11-11 14:12:25--  https://gitlab.com/dstftw/youtube-dl/uploads